In [1]:
import mysql.connector, json, random

# MySQL setup
MYSQL_HOST = "localhost"
MYSQL_USER = "root"
MYSQL_PASSWORD = "naraayana"
DB_NAME = "project_db25_normalized"


In [2]:
# Connect and recreate DB
conn = mysql.connector.connect(host=MYSQL_HOST, user=MYSQL_USER, password=MYSQL_PASSWORD)
cursor = conn.cursor()
cursor.execute(f"DROP DATABASE IF EXISTS {DB_NAME}")
cursor.execute(f"CREATE DATABASE {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")
print(f"🆕 Database created: {DB_NAME}")

🆕 Database created: project_db25_normalized


In [3]:
# =====================
# 1️⃣ CREATE TABLES
# =====================
cursor.execute("""
CREATE TABLE projects (
    project_id VARCHAR(50) PRIMARY KEY,
    title TEXT,
    objective TEXT,
    field VARCHAR(255),
    year VARCHAR(10),
    department VARCHAR(50) DEFAULT 'ISE'
)
""")

cursor.execute("""
CREATE TABLE authors (
    author_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255)
)
""")

cursor.execute("""
CREATE TABLE teams (
    team_id INT AUTO_INCREMENT PRIMARY KEY,
    project_id VARCHAR(50),
    FOREIGN KEY (project_id) REFERENCES projects(project_id) ON DELETE CASCADE
)
""")

cursor.execute("""
CREATE TABLE project_authors (
    id INT AUTO_INCREMENT PRIMARY KEY,
    team_id INT,
    author_id INT,
    FOREIGN KEY (team_id) REFERENCES teams(team_id) ON DELETE CASCADE,
    FOREIGN KEY (author_id) REFERENCES authors(author_id) ON DELETE CASCADE
)
""")

conn.commit()
print("✅ Normalized tables created")


✅ Normalized tables created


In [4]:

# =====================
# 2️⃣ LOAD RAW DATA
# =====================
json_path = "projects-embedding-augmented.json"
names_path = "combined_names.txt"

with open(json_path, "r", encoding="utf-8") as f:
    projects_raw = json.load(f)

with open(names_path, "r", encoding="utf-8") as f:
    author_names = [line.strip() for line in f if len(line.split()) == 2]

print(f"📦 Loaded {len(projects_raw)} projects and {len(author_names)} authors")


📦 Loaded 404 projects and 375 authors


In [5]:

# =====================
# 3️⃣ INSERT AUTHORS
# =====================
cursor.executemany("INSERT INTO authors (name) VALUES (%s)", [(n,) for n in author_names])
conn.commit()

cursor.execute("SELECT author_id FROM authors")
author_ids = [row[0] for row in cursor.fetchall()]
print(f"👨‍🏫 Inserted {len(author_ids)} authors")


👨‍🏫 Inserted 375 authors


In [8]:
import json

for project in projects_raw:
    pid = project.get("project_id")
    title = project.get("title")
    objective = project.get("objective")
    field = project.get("domain")
    year = project.get("year")

    # Skip "_syn" duplicates
    if "_syn" in pid:
        continue

    # Convert lists or dicts to JSON string
    if isinstance(objective, (list, dict)):
        objective = json.dumps(objective)

    cursor.execute("""
        INSERT IGNORE INTO projects (project_id, title, objective, field, year)
        VALUES (%s, %s, %s, %s, %s)
    """, (pid, title, objective, field, year))

    # Teams
    for _ in range(3):
        cursor.execute("INSERT INTO teams (project_id) VALUES (%s)", (pid,))
        team_id = cursor.lastrowid

        selected = random.sample(author_ids, random.randint(2, 3))
        for aid in selected:
            cursor.execute("""
                INSERT INTO project_authors (team_id, author_id) VALUES (%s, %s)
            """, (team_id, aid))

conn.commit()
print("✅ Inserted projects, teams, and team-author links")


✅ Inserted projects, teams, and team-author links


In [9]:

# =====================
# 5️⃣ CREATE VIEWS
# =====================
cursor.execute("""
CREATE OR REPLACE VIEW projects_full AS
SELECT p.project_id, p.title, p.objective, p.field, p.year,
       GROUP_CONCAT(DISTINCT a.name SEPARATOR ', ') AS authors,
       p.department
FROM projects p
JOIN teams t ON p.project_id = t.project_id
JOIN project_authors pa ON t.team_id = pa.team_id
JOIN authors a ON pa.author_id = a.author_id
GROUP BY p.project_id
""")

cursor.execute("""
CREATE OR REPLACE VIEW authors_view AS
SELECT a.name AS author, p.project_id, p.title, p.year
FROM authors a
JOIN project_authors pa ON a.author_id = pa.author_id
JOIN teams t ON pa.team_id = t.team_id
JOIN projects p ON t.project_id = p.project_id
""")

cursor.execute("""
CREATE OR REPLACE VIEW teams_view AS
SELECT t.team_id, p.project_id, p.title,
       GROUP_CONCAT(a.name SEPARATOR ', ') AS authors
FROM teams t
JOIN project_authors pa ON t.team_id = pa.team_id
JOIN authors a ON pa.author_id = a.author_id
JOIN projects p ON t.project_id = p.project_id
GROUP BY t.team_id
""")

conn.commit()
print("✅ Views created")


✅ Views created


In [10]:

# =====================
# 6️⃣ PREVIEW OUTPUT
# =====================
print("\n🔍 SAMPLE PROJECTS_FULL:")
cursor.execute("SELECT * FROM projects_full LIMIT 3")
for row in cursor.fetchall():
    print(row)

print("\n🔍 SAMPLE TEAMS:")
cursor.execute("SELECT * FROM teams_view LIMIT 3")
for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()
print("\n✅ Normalized database build complete.")


🔍 SAMPLE PROJECTS_FULL:
('auto001', 'Instagram Reach Analysis', 'Analyze Instagram reach data to identify factors influencing post visibility using Python libraries.', 'Artificial Intelligence / Data Science', '2023', 'Aaditya Dahlan, Achyut Rout, Akhya Purwar, Arihant Dagade, Arushi Maddeshiya, Ashal Fathima, Ashal Puri, Ashish Tiwari, Aviral Doctor, Azmi Cauhan, Chandranshu Dinu, Charita Kakar, Deeksha Jajoo, Ehtisham Taneja, Elango Prajapati, Gurbaksh Khaliq, Hansh Mehendale, Harbans Barai, Iditri Wagh, Inabat Dhindhwal, Jami Kanchapu, Jasbir Maheshwari, Jasbir Roy, Jyotsana Nagi, Kirandeep Krish, Kishan Nanwani, Konkana Dayal, Layth Dave, Mahit Kanda, Nandini Shaikh, Nav Borah, Navya Thakur, Nimesh Shankdhar, Nivedita Saraswat, Palak Asthana, Paramjit Bhatia, Prasanna Chowdhury, Priya Dohrey, Priya Hayer, Rizwa Shahid, Sangeeta Baiswar, Sanjeeda Lala, Sanskar Navas, Sanskar Sura, Satyarth Krishnan, Shagufta Samdharshni, Shaniya Brahmbhatt, Shikha Chandra, Sudhakar Souza, Tanya Pra